In [2]:
# google driva bağlanma
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [8]:
import numpy as np
import cv2
import os
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [9]:

import os
print(os.listdir("/gdrive/My Drive/Banana_Ripeness_Classification"))


['unripe', 'ripe', 'rotten', 'overripe']


In [10]:
# Dizini ve kategorileri ayarlama
DATADIR = '/gdrive/My Drive/Banana_Ripeness_Classification/'
CATEGORIES = ["overripe", "ripe", "rotten", "unripe"]
IMG_SIZE = 100

In [11]:
# Veri kümesi oluşturma işlevi
def create_all_data():
    all_data = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                all_data.append([new_array, class_num])
            except Exception as e:
                pass
    return all_data

In [12]:
# Veri kümesini oluşturma
all_data = create_all_data()

In [13]:
print("Toplam Görüntü Sayısı: ", len(all_data))

Toplam Görüntü Sayısı:  13478


In [14]:
X = []
y = []

for features, label in all_data:
    X.append(features)
    y.append(label)

In [15]:
# Senaryo 1: %80 Train, %20 Test ve %10 Validation
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state =42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [16]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [17]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(9703, 100, 100, 3)
(2696, 100, 100, 3)
(1079, 100, 100, 3)
(9703,)
(2696,)
(1079,)


In [18]:
# Etiketleri kategorik hale dönüştürme
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)
y_val = to_categorical(y_val, num_classes=4)

In [19]:
# ImageNet verileri üzerinde önceden eğitilmiş VGG16 modelini yükleme
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False


In [20]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

In [21]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
# Modeli eğitimi
history = model.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_val, y_val))


Epoch 1/50
152/152 [==============================] - 325s 2s/step - loss: 2.9645 - accuracy: 0.8770 - val_loss: 0.2414 - val_accuracy: 0.9435
Epoch 2/50
152/152 [==============================] - 296s 2s/step - loss: 0.3607 - accuracy: 0.9264 - val_loss: 0.1645 - val_accuracy: 0.9611
Epoch 3/50
152/152 [==============================] - 320s 2s/step - loss: 0.2340 - accuracy: 0.9456 - val_loss: 0.1462 - val_accuracy: 0.9694
Epoch 4/50
152/152 [==============================] - 296s 2s/step - loss: 0.1806 - accuracy: 0.9557 - val_loss: 0.1440 - val_accuracy: 0.9685
Epoch 5/50
152/152 [==============================] - 297s 2s/step - loss: 0.1516 - accuracy: 0.9609 - val_loss: 0.1682 - val_accuracy: 0.9685
Epoch 6/50
152/152 [==============================] - 316s 2s/step - loss: 0.1495 - accuracy: 0.9629 - val_loss: 0.1383 - val_accuracy: 0.9639
Epoch 7/50
152/152 [==============================] - 296s 2s/step - loss: 0.1662 - accuracy: 0.9590 - val_loss: 0.1479 - val_accuracy: 0.9639

In [23]:
# Modeli değerlendirme
score_train = model.evaluate(x_train, y_train)
score_test = model.evaluate(x_test, y_test)
score_val = model.evaluate(x_val, y_val)
print("Train Accuracy:", score_train[1])
print("Test Accuracy:", score_test[1])
print("Validation Accuracy:", score_val[1])

34/34 [==============================] - 29s 861ms/step - loss: 0.2172 - accuracy: 0.9805
Train Accuracy: 0.9985571503639221
Test Accuracy: 0.9595697522163391
Validation Accuracy: 0.9805375337600708


In [24]:
# Test setini kullanarak tahmin edme ve değerlendirme
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

85/85 [==============================] - 74s 859ms/step


In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
# Metrikleri hesaplama
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Karışıklık Matrisi ve Sınıflandırma Raporu
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes)


In [30]:
# Sonuçları görüntüleme
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

Accuracy: 0.9595697329376854
Precision: 0.9596944953547384
Recall: 0.9595697329376854
F1 Score: 0.9595231835597621

Confusion Matrix:
 [[504  32   8   1]
 [  7 764  20   0]
 [ 14   8 874   6]
 [  0   0  13 445]]

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.92      0.94       545
           1       0.95      0.97      0.96       791
           2       0.96      0.97      0.96       902
           3       0.98      0.97      0.98       458

    accuracy                           0.96      2696
   macro avg       0.96      0.96      0.96      2696
weighted avg       0.96      0.96      0.96      2696

